In [13]:
##############################
## CREATE COUNTABLE BINARY ARRAY
#############################

import numpy as np
import scipy.misc as smp
import random



arr_size_half = 20
max_N = 3
view_size_half = 0

def create_inp_outp_array():
    max_objects = 3
    n_ones = random.randint(1, max_objects)
    one_positions = random.sample(range(view_size_half, view_size_half + max_objects), n_ones)
    binary_array = np.zeros(max_objects+2*view_size_half)
    binary_array[one_positions] = 255
    return binary_array, n_ones
    

 

def make_data_set(n_samples):
  mult_inp, mult_out = create_inp_outp_array()
  for i in range(n_samples - 1):
      single_inp, single_out = create_inp_outp_array()
      mult_inp = np.vstack([mult_inp, single_inp])
      mult_out = np.vstack((mult_out, single_out))
  return mult_inp, mult_out



train_input, train_output = make_data_set(1000)
max_N=20
test_input, test_output = make_data_set(1000)


#Print single one
inp_ex, out_ex = make_data_set(20)
print("inp_ex", inp_ex)
print("out_ex", out_ex)


#Print set data
print("===========================")
print("train_input: number of examples: ", train_input.shape)
print("train_input: number of examples: ", train_input[:,0].size)
print("train_input: single example size: ", train_input[0].size)
print("train_input: dim of elements in example: ", train_input[0,0].size)

inp_ex [[255.   0. 255.]
 [  0.   0. 255.]
 [255.   0. 255.]
 [  0.   0. 255.]
 [  0.   0. 255.]
 [255. 255. 255.]
 [  0.   0. 255.]
 [  0. 255. 255.]
 [255. 255.   0.]
 [255. 255.   0.]
 [255. 255.   0.]
 [255.   0.   0.]
 [255. 255. 255.]
 [  0. 255. 255.]
 [  0.   0. 255.]
 [255.   0. 255.]
 [  0.   0. 255.]
 [255. 255. 255.]
 [255. 255. 255.]
 [255.   0.   0.]]
out_ex [[2]
 [1]
 [2]
 [1]
 [1]
 [3]
 [1]
 [2]
 [2]
 [2]
 [2]
 [1]
 [3]
 [2]
 [1]
 [2]
 [1]
 [3]
 [3]
 [1]]
train_input: number of examples:  (1000, 3)
train_input: number of examples:  1000
train_input: single example size:  3
train_input: dim of elements in example:  1


In [20]:
#!/usr/bin/env python

#import gym
import numpy as np
import numpy.random as rd
import matplotlib.pyplot as plt
#from gym.envs.classic_control.cartpole import CartPoleEnv
#from cartpole_utils import plot_results,print_results
import tensorflow as tf
import copy
import matplotlib
import matplotlib.pyplot as plt


def init_weights(shape, name):
    return tf.Variable(tf.random_normal(shape, stddev=0.01)/ np.sqrt(dim_state), name=name)

# This network is the same as the previous one except with an extra hidden layer + dropout
def Q_network(state_holder, w_h, w_h2, w_o, b):
    h = tf.nn.relu(tf.matmul(state_holder, w_h))
    h2 = tf.nn.relu(tf.matmul(h, w_h2))
    a_z = tf.matmul(h2, w_o, name='output_activation') + b
    Q = tf.nn.sigmoid(a_z, name='Q_model')
    return Q


########################
## SET PARAMETERS
########################
# Algorithm parameters

step_reward = +0
final_reward = +1

learning_rate = 3e-3
gamma = 1.0
epsilon = 1.
epsi_decay = .995
lr_decay = .999
epsilon_min = 0.01
learning_min = 0.00001


# Load Input=Start-of-a-game &&  Classes=Check-for-reward
trX = train_input
trY = train_output
teX = test_input
teY = test_output

print(trX.shape)
print(trY.shape)
print(teX.shape)
print(teY.shape)

## SET ENVIRONMENT + #ofActions #ofStates
dim_state = trX[0,:].size
n_action = trX[0,:].size



# General parameters
render = False
N_print_every = 100
N_trial = trX[:,0].size
N_trial_test = N_trial = teX[:,0].size








##############################
## BUILD NETWORK
#############################



# PLACEHOLDER
action_holder = tf.placeholder(dtype=tf.int32, name="Y")  # +1 because that is the node responsible to say if it is done ore not
state_holder = tf.placeholder(dtype=tf.float32, shape=(1, dim_state), name='symbolic_state')
next_state_holder = tf.placeholder(dtype=tf.float32, shape=(1, dim_state), name='symbolic_state')

# Initialize the parameters of the Q model
n_h_neurons = 600
w_h = init_weights([trX[0,:].size, n_h_neurons], "w_h")
w_h2 = init_weights([n_h_neurons, n_h_neurons], "w_h2")
w_o = init_weights([n_h_neurons, n_action], "w_o")
b0 = np.zeros(n_action , dtype=np.float32)
b = tf.Variable(initial_value=b0, trainable=True, name='bias')

Q = Q_network(state_holder, w_h, w_h2, w_o,b)
next_Q = Q_network(next_state_holder, w_h, w_h2, w_o,b)



### COMPARE REWARDS ####
#PLACEHOLDER
r_holder = tf.placeholder(dtype=tf.float32, name='symbolic_value_estimation')
is_done_holder = tf.placeholder(dtype=tf.float32, name='is_done')

next_action_Q_max = tf.to_int32(tf.argmax(next_Q, dimension=1, name='next_step_maxQ_action'))           # [ 0.1   0.4   0.8  0.8] -->  3, 4
# next_action_holder = tf.placeholder(dtype=tf.int32, name='symbolic_next_action')
R = Q[0, action_holder]
next_R = r_holder + gamma * next_Q[0, next_action_Q_max[0]] * (1 - is_done_holder)
error = (R - next_R)**2

# Define the operation that performs the optimization
learning_rate_holder = tf.placeholder(dtype=tf.float32, name='symbolic_state')
training_step = tf.train.GradientDescentOptimizer(learning_rate_holder).minimize(error)

sess = tf.Session()  # FOR NOW everything is symbolic, this object has to be called to compute each value of Q
sess.run(tf.initialize_all_variables())

################################
################################





################################
## PI(S)
## First check if its random time THEN
        # - calculate State--Network--> Q
        # - take maximum of Q
        # - just remember and return index of maximum    ---> returns ACTION that leads to max_Q-value (not value itself)
################################

def policy(state):
    if rd.rand() < epsilon:
        return rd.randint(0, n_action)
    reshaped_state = state.reshape(1, dim_state)
    Q_values = sess.run(Q, feed_dict={state_holder: state.reshape(1, dim_state)})
    Q_values_reduced = Q_values[0, :]
    val = np.max(Q_values[0, :])                                                                                      #Change in dimension
    max_indices = np.where(Q_values[0, :] == val)[0]
    push_finish_button = False
    #print("Q_values[0,-1]", Q_values[0,-1])
    #if(Q_values[0,-1]>0.0):
        #push_finish_button = True

    return rd.choice(max_indices), push_finish_button


# define Action
def draw_point(state, action, finish_button,internal_counter_memory):    #later for convolutoinal NN --> state = matrix rank-2 like a real image is
    reward = 0
    state_copy = copy.copy(state)
    #print("draw point")
    #if(finish_button==False):
        #print("finish_button = False")
    if(state[action]==255):
        #print("state[action]==1")
        #pass
        #print("drew on white")
        reward += step_reward                      #Intermediate reward: giving points if set point on white area
        internal_counter_memory += 1
    state_copy[action] = 0

    #print("internal_counter_memory", internal_counter_memory)
    return state_copy, reward, internal_counter_memory



########################
## Get maximum Q_VALUE (not action)
#######################
def Q_max(state):
    Q_values = sess.run(Q, feed_dict={state_holder: state.reshape(1, dim_state)})
    return Q_values.max()






##############################
## ACTUAL RUN
###########################

time_list = []
reward_list = []
err_list = []
val_list = []
mean_reward = 0

print("Here are the",  trX[:, 0].size ," objects and classes to learn")
for l in range(trX[:, 0].size):
    #print(l,"th object: ", trX[l,:])
    #print(l,"th class : ", trY[l, :])
    pass

mean_reward_list = []
n_episodes = trX[:, 0].size
for k in range(n_episodes):

    #print("N-th episode: ", k)
    acc_reward = 0  # Init the accumulated reward
    k_n = rd.randint(0, trX[:,0].size)                                            ######   !!!!!!!!!!!!!!!!!

    if(epsilon>epsilon_min):
        epsilon *= epsi_decay
    if (learning_rate > learning_min):
        learning_rate *= lr_decay

    '''
    if k > N_trial+1:
        epsilon = 0
        learning_rate = 0
        observation = teX[k-N_trial, :]
        classy = teY[k-N_trial, :]
    else:
        observation = copy.copy(trX[k, :])
        classy = trY[k, :]
    '''
    observation = copy.copy(trX[k_n, :])
    classy = trY[k_n, :]
    if(k%50<0):
        print("Observation:", observation)

    #("observation: ", observation)
    #print("classy: ", classy)
    #print("-----------------")

    trial_err_list = []
    internal_counter_memory = 0
    trial_duration = 15
    push_finish_button = False

    for t in range(trial_duration):  # The number of time steps in this game is maximum 200

        #marker, push_finish_button = policy(observation)  # Init the first action
        ###########################
        if(t==classy):
            push_finish_button=True

            if (t == classy):
                mean_reward += acc_reward
                if (k % 50 == 0):
                    mean_reward = float(mean_reward) / 50.0
                    print(k, "th mean_reward=", mean_reward)
                    mean_reward_list.append(mean_reward)
                    mean_reward = 0.0
                break  # Stop the trial when the environment says it is done
            '''
        if(k>1000 and k<1005 ):
            print("Before Everything is calculated")
            print("t=", t)
            print("old_state= ", observation)
        '''

        old_observation = copy.copy(observation)
        old_internal_counter_memory = copy.copy(internal_counter_memory)

        Q_values = sess.run(Q, feed_dict={state_holder: observation.reshape(1, dim_state)})
        Q_values_reduced = Q_values[0, :]
        old_Q_values_reduced = copy.copy(Q_values_reduced)
        val = np.max(Q_values_reduced)  # Change in dimension
        max_indices = np.where(Q_values_reduced == val)[0]
        #push_finish_button = False
        #print("Q_values[0,-1]", Q_values[0, -1])
        #if (Q_values[0, -1] < -0.5):
            #push_finish_button = True

        action = rd.choice(max_indices)
        if rd.rand() < epsilon:
            action = rd.randint(0, n_action)

        reward = 0
        #print("internal_counter_memory before: ", internal_counter_memory)
        new_observation, reward, internal_counter_memory = draw_point(observation, action, push_finish_button,internal_counter_memory )
        #print("internal_counter_memory after: ", internal_counter_memory)


        if(t==classy-1):
            #Create one-hot-vector from counter
            if (internal_counter_memory == classy):
                reward += final_reward #10
        #print("reward: ", reward)

        err = sess.run(error, feed_dict={
            state_holder: observation.reshape(1, dim_state),
            next_state_holder: new_observation.reshape(1, dim_state),
            action_holder: action,
            is_done_holder: np.float32(push_finish_button),
            r_holder: reward})

        sess.run(training_step, feed_dict={
            state_holder: observation.reshape(1, dim_state),
            next_state_holder: new_observation.reshape(1, dim_state),
            action_holder: action,
            is_done_holder: np.float32(push_finish_button),
            r_holder: reward,
            learning_rate_holder: learning_rate})

        # Compute the Bellman Error for monitoring
        err2 = sess.run(error, feed_dict={
            state_holder: observation.reshape(1, dim_state),
            next_state_holder: new_observation.reshape(1, dim_state),
            action_holder: action,
            is_done_holder: np.float32(push_finish_button),
            r_holder: reward})
        trial_err_list.append(err)

            #keep track of old observation for print
            #if(k>1000 and k<1005 ):
             #   print("Everything is calculated")
              #  print("t=", t)
               # print("old_state= ", observation)

        Q_values = sess.run(Q, feed_dict={state_holder: old_observation.reshape(1, dim_state)})
        Q_values_reduced = Q_values[0, :]

        observation = new_observation  # Pass the new state to the next step
        acc_reward += reward  # Accumulate the reward
        # Add the error in a trial-specific list of errors


        weights = sess.run(w_o, feed_dict={state_holder: observation.reshape(1, dim_state)})


        #if((k>=0 and k<5) or (k%100==0) ):
        if ((k >= 40 and k < 20)):
            if(t==0):
                print("=======================")
                print("==== K= ", k, "========")
            print("t=", t)
            print("old_state= ", old_observation)
            print("old_internal_counter_memory",old_internal_counter_memory)
            print("Q_values before: ", old_Q_values_reduced)
            print("action= ", action)
            print("Q_values after: ", Q_values_reduced)
            print("state= ", observation)
            print("internal_counter_memory", internal_counter_memory)
            print("acc_reward= ", acc_reward)
            print("orig nr. of 1s", classy)
            print("w_o= ", weights[0,0])
            print("------End of trial:")
            print("Predicted class: ", internal_counter_memory)
            print("Real class:      ", classy)
            print("err1 ", err)
            print("err2 ", err2)
            print("---------end of live action -------- \n \n")


        

        '''
        if(k%1000==0 and push_finish_button):
        #if (k < 50 and push_finish_button):
            print("k=",k)
            print("mean_reward=", float(mean_reward)/1000.0)
            print("Last t= ", internal_counter_memory)
            print("acc_reward", acc_reward)
            print("Nth trial", k)
            print("State= ", observation)
            print("Class= ", classy)
            print("Action= ", action)
            #print("w_o= ", weights[0, 0])
            #print("error",err)
            print("==============================")
            mean_reward = 0
        '''



        #internal_counter_memory += 1




    # Stack values for monitoring

    err_list.append(np.mean(trial_err_list))
    time_list.append(t + 1)
    reward_list.append(acc_reward)  # Store the result
    #print("acc_reward", acc_reward)


print("mean_reward_list", mean_reward_list)
reward_file = 'Mean_rewards/'  + str(n_h_neurons) + '_neurons_' + str(n_action) + '_max_objects'
np.save(reward_file, mean_reward_list)


matplotlib.get_backend()
#print(acc_arr.shape)
#print(episodes.shape)
#print("np.arange(0,n_episodes)", np.arange(0,n_episodes))
legend_string = str(n_h_neurons) + " neurons/hidden layer"
plt.plot(np.arange(0,len(mean_reward_list)), mean_reward_list, label = legend_string)
plt.legend()
print("legend_string= ", legend_string)
#plt.legend(legend_string,)
plt.ylabel('Success rate')
plt.xlabel('Episodes')
plt.ylim(0,1)
title_string = 'Success rate for agent when counting binaries between 0 and ' + str(n_action) + ' binaries'
plt.title(title_string)
plt.savefig('Q_learning_binaries.png')
plt.show()


(1000, 3)
(1000, 1)
(1000, 3)
(1000, 1)
Here are the 1000  objects and classes to learn
0 th mean_reward= 0.0
50 th mean_reward= 0.16
100 th mean_reward= 0.44
150 th mean_reward= 0.58
200 th mean_reward= 0.72
250 th mean_reward= 0.8
300 th mean_reward= 0.82
350 th mean_reward= 0.8
400 th mean_reward= 0.88
450 th mean_reward= 0.88
500 th mean_reward= 0.96
550 th mean_reward= 0.9
600 th mean_reward= 0.94
650 th mean_reward= 1.0
700 th mean_reward= 0.98
750 th mean_reward= 1.0
800 th mean_reward= 0.96
850 th mean_reward= 1.0
900 th mean_reward= 1.0
950 th mean_reward= 1.0
mean_reward_list [0.0, 0.16, 0.44, 0.58, 0.72, 0.8, 0.82, 0.8, 0.88, 0.88, 0.96, 0.9, 0.94, 1.0, 0.98, 1.0, 0.96, 1.0, 1.0, 1.0]


FileNotFoundError: ignored